In [6]:
import pandas as pd
# Download and import tokenizer libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [7]:
df = pd.read_csv('/Users/jeanphilippepetit-frere/Python_projects/Quotes_classifier/quotes.csv')

In [8]:
df

,Unnamed: 0,quote,author,tags
0,0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"['be-yourself', 'gilbert-perreira', 'honesty',..."
1,1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"['best', 'life', 'love', 'mistakes', 'out-of-c..."
2,2,“Two things are infinite: the universe and hum...,Albert Einstein,"['human-nature', 'humor', 'infinity', 'philoso..."
3,3,"“So many books, so little time.”",Frank Zappa,"['books', 'humor']"
4,4,“A room without books is like a body without a...,Marcus Tullius Cicero,"['books', 'simile', 'soul']"
...,...,...,...,...
2503,2995,“Morality is simply the attitude we adopt towa...,"Oscar Wilde,","['morality', 'philosophy']"
2504,2996,“Don't aim at success. The more you aim at it ...,"Viktor E. Frankl,","['happiness', 'success']"
2505,2997,"“In life, finding a voice is speaking and livi...",John Grisham,['inspirational-life']
2506,2998,"“Winter is the time for comfort, for good food...",Edith Sitwell,"['comfort', 'home', 'winter']"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2508 entries, 0 to 2507
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2508 non-null   int64 
 1   quote       2508 non-null   object
 2   author      2508 non-null   object
 3   tags        2508 non-null   object
dtypes: int64(1), object(3)
memory usage: 78.5+ KB


In [10]:
#use nlp and clustering on the dataset
#gridsearch

In [11]:
# Stopword function
def stopword(string):
    from nltk.corpus import stopwords
    res = [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(res)

In [12]:
# Initialize lemmatizer
wl = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [13]:
# Tokenize Summaries
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    res = [wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(res)

In [14]:
# Text pre-processing
def get_pre_preprocessing(df1,col):
    import pandas as pd
    import re
    from nltk.tokenize import RegexpTokenizer
    import numpy as np
    df = df1.copy()
# Convert Summary data to type(str)
    df[col] = df[col].apply(lambda x: str(x))
# Remove html tags and leading 'b's
    df[col] = df[col].apply(lambda row: re.compile(r'<.*?>').sub(r' ',row))
# Remove special characters, replace with space
    df[col] = df[col].apply(lambda row: re.compile(r'\W').sub(r' ',row))
# Remove single characters
    df[col] = df[col].apply(lambda row: re.compile(r'\s+[a-zA-Z]\s+').sub(r' ',row))
# Remove stil characters from start
    df[col] = df[col].apply(lambda row: re.compile(r'\^[a-zA-Z]\s+').sub(r' ',row))
# Clean up multi-space gaps
    df[col] = df[col].apply(lambda row: re.compile(r'\s+').sub(r' ',row))
# Converting to lower case
    df[col] = df[col].apply(lambda row: row.lower())
# Apply stopwords func to Summaries
    df[col] = df[col].apply(lambda text: stopword(text))
# Apply lemmatizer func to Summaries
    df[col] = df[col].apply(lambda text: lemmatizer(text))
# Initialize tokenizer and tokenize summary data
    tokenizer = RegexpTokenizer(r'\w+')
#apply tokinizer to text data
    df[col] = df[col].apply(tokenizer.tokenize)
# Create token length count column
    df['token_count'] = [len(x) for x in df[col]]
# Calculate standard deviation of token counts
    print(pd.DataFrame.std(df.token_count, ddof=2))
# Checking standard deviation using np
    arr = pd.Series.to_numpy(df.token_count)
    print(np.std(arr, ddof=2))
#add the tokenize column to the original column 
    df1['token'] = df[col]
    return df1

In [15]:
df1 = get_pre_preprocessing(df,'quote')
df1

23.004741780685855
23.004741780685855


,Unnamed: 0,quote,author,tags,token
0,0,“Be yourself; everyone else is already taken.”,Oscar Wilde,"['be-yourself', 'gilbert-perreira', 'honesty',...","[everyone, else, already, take]"
1,1,"“I'm selfish, impatient and a little insecure....",Marilyn Monroe,"['best', 'life', 'love', 'mistakes', 'out-of-c...","[selfish, impatient, little, insecure, make, m..."
2,2,“Two things are infinite: the universe and hum...,Albert Einstein,"['human-nature', 'humor', 'infinity', 'philoso...","[two, thing, infinite, universe, human, stupid..."
3,3,"“So many books, so little time.”",Frank Zappa,"['books', 'humor']","[many, book, little, time]"
4,4,“A room without books is like a body without a...,Marcus Tullius Cicero,"['books', 'simile', 'soul']","[room, without, book, like, body, without, soul]"
...,...,...,...,...,...
2503,2995,“Morality is simply the attitude we adopt towa...,"Oscar Wilde,","['morality', 'philosophy']","[morality, simply, attitude, adopt, towards, p..."
2504,2996,“Don't aim at success. The more you aim at it ...,"Viktor E. Frankl,","['happiness', 'success']","[aim, success, aim, make, target, go, miss, su..."
2505,2997,"“In life, finding a voice is speaking and livi...",John Grisham,['inspirational-life'],"[life, find, voice, speak, live, truth, origin..."
2506,2998,"“Winter is the time for comfort, for good food...",Edith Sitwell,"['comfort', 'home', 'winter']","[winter, time, comfort, good, food, warmth, to..."


In [ ]:
#do some sentiment analysis
#https://www.datacamp.com/community/tutorials/simplifying-sentiment-analysis-python 